In [8]:
import numpy as np
import pandas as pd
import os
import xlrd
import re

In [9]:
excel_files = [f for f in os.listdir("data")]

In [10]:
file = 'RDO No. 1 - Laoag City, Ilocos Norte.xls'
test = pd.ExcelFile(f'data/{file}')

In [11]:
sheet_names = sorted([name for name in test.sheet_names if name.strip().lower().startswith('sheet')], key=lambda name: int(re.search(r'\d+', name).group()))
last_sheet_name = sheet_names[-1] if sheet_names else None

In [12]:
last_sheet_name

'Sheet 9 (DO 047-2023)'

In [13]:
df = pd.read_excel("data/"+file, sheet_name=last_sheet_name, header=None)

In [14]:
df

,0,1,2,3
0,Republic of the Philippines,NaN,NaN,NaN
1,DEPARTMENT OF FINANCE,NaN,NaN,NaN
2,MANILA,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,DEPARTMENT ORDER NO. 047 - 2023,NaN,NaN,NaN
...,...,...,...,...
25341,TAXES SHALL BE (1) THE FAIR MARKET VALUE ...,NaN,NaN,NaN
25342,(i.e. ZONAL VALUES) OR (2.) THE FAIR MAR...,NaN,NaN,NaN
25343,"PROVINCIAL/CITY/MUNICIPAL ASSESSOR, WHICH...",NaN,NaN,NaN
25344,NaN,NaN,NaN,NaN


In [15]:
# FINAL RN
def test_function():
    print("All conditions met! Function called.\n")

for index, row in df.iterrows():
    combined_row = ''.join(map(str, row.dropna())).strip()
    
    # Initialize flags
    province_match_flag = False
    city_match_flag = False
    barangay_match_flag = False
    
    # Search for "Province :" pattern
    if re.search(r"province\s*:", combined_row, re.IGNORECASE):
        province_match_flag = True
        print(f"Province match found in row {index}: {combined_row}")
    
    # Check if the next row has "City" or "Municipality"
    if index + 1 < len(df):
        next_row = ''.join(map(str, df.iloc[index + 1].dropna())).strip()
        if re.search(r"\b(City|Municipality)\b", next_row, re.IGNORECASE):
            city_match_flag = True
            print(f"City / Municipality match found in row {index + 1}: {next_row}")
    
    # Check if the row after that has "Barangay" or "Zone"
    if index + 2 < len(df):
        following_row = ''.join(map(str, df.iloc[index + 2].dropna())).strip()
        if re.search(r"\b(Barangay|Zone)\b", following_row, re.IGNORECASE):
            barangay_match_flag = True
            print(f"Barangay / Zone match found in row {index + 2}: {following_row}")
    
    # Call the function only if all conditions are true
    if province_match_flag and city_match_flag and barangay_match_flag:
        test_function()
        #then skip 2 rows to make more efficient somehow


City / Municipality match found in row 15: LAOAG CITY,  ILOCOS NORTE,  REVENUE  REGION  NO.  1 - CALASIAO,
Barangay / Zone match found in row 23: determine the  fair   market   value of   real   properties   located   in  each  zone   or   areas,    subject  to   automatic
City / Municipality match found in row 31: District Office No. 1 - Laoag City, Ilocos Norte, Revenue Region No. 1 - Calasiao, Pangasinan.
City / Municipality match found in row 36: marker value as shown in the schedule of values of the provincial or city assessor and (2) the gross selling
City / Municipality match found in row 118: Revenue District Office No. 1 - LAOAG CITY, ILOCOS NORTE
Province match found in row 120: Province :ILOCOS NORTE
City / Municipality match found in row 121: Municipality :LAOAG CITY
Barangay / Zone match found in row 122: Barangay / Zone :NO. 1 - SAN LORENZO (POBLACION)DO No. 047 - 2023
All conditions met! Function called.

City / Municipality match found in row 160: Revenue District Offic

In [16]:
def test_function():
    print("All conditions met! Function called.\n")

count = 0
for index, row in df.iterrows():
    combined_row = ''.join(map(str, row.dropna())).strip()
    
    # Initialize flags
    province_match_flag = False
    city_match_flag = False
    barangay_match_flag = False
    additional_info_flag = False
    
    # Search for "Province :" pattern
    province_pattern = re.search(r"province\s*:\s*(.*)", combined_row, re.IGNORECASE)
    if province_pattern:
        province_match_flag = True
        province_value = province_pattern.group(1).strip()  # Capture the text after "Province :"
        print(f"Province match found in row {index}: {province_value}")
    
    # Initialize city and barangay values
    city_value = None
    barangay_value = None
    
    # Check if the next row has "City" or "Municipality"
    if index + 1 < len(df):
        next_row = ''.join(map(str, df.iloc[index + 1].dropna())).strip()
        city_pattern = re.search(r"\b(City|Municipality)\s*:\s*(.*)", next_row, re.IGNORECASE)
        if city_pattern:
            city_match_flag = True
            city_value = city_pattern.group(2).strip()  # Capture the text after "City :" or "Municipality :"
            print(f"City / Municipality match found in row {index + 1}: {city_value}")
    
    # Check if the row after that has "Barangay" or "Zone"
    if index + 2 < len(df):
        following_row = ''.join(map(str, df.iloc[index + 2].dropna())).strip()
        
        # Modified pattern to handle "DO No." as part of another word
        barangay_pattern = re.search(r"\b(Barangay|Zone)\s*:\s*([^\n\r]*)(?=DO\s+No|$)", following_row, re.IGNORECASE)
        
        if barangay_pattern:
            barangay_match_flag = True
            barangay_value = barangay_pattern.group(2).strip()  # Capture the text after "Barangay :" or "Zone :"
            print(following_row)
            print(f"Barangay / Zone match found in row {index + 2}: {barangay_value}")

                
    # Check specific columns for additional information in the next 3 rows
    for offset in range(3, 6):
        if index + offset < len(df):
            subsequent_row = df.iloc[index + offset]
            
            # Check each column independently
            first_column_match = re.search(r"(STREET NAME|SUBDIVISION|CONDOMINIUM)", str(subsequent_row.iloc[0]), re.IGNORECASE)
            second_column_match = re.search(r"V\s*I\s*C\s*I\s*N\s*I\s*T\s*Y|VICINITY", str(subsequent_row.iloc[1]), re.IGNORECASE)
            third_column_match = re.search(r"C.*L.*A.*S.*S.*I.*F.*I.*C.*A.*T.*I.*O.*N", str(subsequent_row.iloc[2]), re.IGNORECASE | re.DOTALL)
            fourth_column_match = re.search(r"REV\s*ZV\s*/\s*SQM", str(subsequent_row.iloc[3]), re.IGNORECASE)

            # Set the flag if all columns match the criteria
            if first_column_match and second_column_match and third_column_match and fourth_column_match:
                additional_info_flag = True
                print(f"Additional information match found in row {index + offset}")
                # count += 1 # wrong place to put count
                break  # No need to check further if a match is found

    # Call the function only if all conditions are true
    if province_match_flag and city_match_flag and barangay_match_flag and additional_info_flag:
        test_function()
        count += 1 #count should be here
        
    # REMOVE THIS!!!!!!!!!!!
    if count > 10:
        break

print(f'count: {count}')

Additional information match found in row 124
Province match found in row 120: ILOCOS NORTE
City / Municipality match found in row 121: LAOAG CITY
Barangay / Zone :NO. 1 - SAN LORENZO (POBLACION)DO No. 047 - 2023
Barangay / Zone match found in row 122: NO. 1 - SAN LORENZO (POBLACION)DO No. 047 - 2023
Additional information match found in row 124
All conditions met! Function called.

Additional information match found in row 124
Additional information match found in row 166
Province match found in row 162: ILOCOS NORTE
City / Municipality match found in row 163: LAOAG CITY
Barangay / Zone :NO. 1 - SAN LORENZO (POBLACION) -- continuedDO No. 047 - 2023
Barangay / Zone match found in row 164: NO. 1 - SAN LORENZO (POBLACION) -- continuedDO No. 047 - 2023
Additional information match found in row 166
All conditions met! Function called.

Additional information match found in row 166
Additional information match found in row 191
Province match found in row 187: ILOCOS NORTE
City / Municipalit

In [17]:
following_row = "Barangay / Zone :NO. 1 - SAN LORENZO (POBLACION)DO No. 047 - 2023"

# Modified pattern to handle "DO No." as part of another word
barangay_pattern = re.search(r"\b(Barangay|Zone)\s*:\s*([^\n\r]*?)(?=\s*DO\s+No|$)", following_row, re.IGNORECASE)
if barangay_pattern:
    barangay_match_flag = True
    barangay_value = barangay_pattern.group(2).strip()  # Capture the text after "Barangay :" or "Zone :"
    print(following_row)
    print(barangay_value)

Barangay / Zone :NO. 1 - SAN LORENZO (POBLACION)DO No. 047 - 2023
NO. 1 - SAN LORENZO (POBLACION)


In [18]:
following_row = ''.join(map(str, df.iloc[index + 2].dropna())).strip()

# Updated pattern to stop capturing before "DO No."



In [19]:
provinces = []
cities = []
barangays = []

In [20]:
def test_function():
    print("All conditions met! Function called.\n")

index = 0
count = 0
while index < len(df):
    row = df.iloc[index]
    combined_row = ''.join(map(str, row.dropna())).strip()
    
    # Initialize flags
    province_match_flag = False
    city_match_flag = False
    barangay_match_flag = False
    additional_info_flag = False
    
    # Search for "Province :" pattern
    province_pattern = re.search(r"province\s*(.*)", combined_row, re.IGNORECASE)
    if province_pattern:
        province_match_flag = True
        province_value = province_pattern.group(1).strip()  # Capture the text after "Province :"
    
    city_value = None    
    # Check if the next row has "City" or "Municipality"
    if index + 1 < len(df):
        next_row = ''.join(map(str, df.iloc[index + 1].dropna())).strip()
        city_pattern = re.search(r"\b(City|Municipality)\s*\s*(.*)", next_row, re.IGNORECASE)
        if city_pattern:
            city_match_flag = True
            city_value = city_pattern.group(2).strip()  # Capture the text after "City :" or "Municipality :"
            
    barangay_value = None
    # Check if the row after that has "Barangay" or "Zone"
    if index + 2 < len(df):
        following_row = ''.join(map(str, df.iloc[index + 2].dropna())).strip()
        barangay_pattern = re.search(r"\b(Barangay|Zone)\s*\s*(.*)", following_row, re.IGNORECASE)
        if barangay_pattern:
            barangay_match_flag = True
            barangay_value = barangay_pattern.group(2).strip()  # Capture the text after "Barangay :" or "Zone :"
            
            
    if province_match_flag and city_match_flag and barangay_match_flag:
        # Check specific columns for additional information in the next 3 rows
        for offset in range(3, 6):
            if index + offset < len(df):
                subsequent_row = df.iloc[index + offset]
                
                # Check each column independently
                first_column_match = re.search(r"(STREET NAME|SUBDIVISION|CONDOMINIUM)", str(subsequent_row.iloc[0]), re.IGNORECASE)
                second_column_match = re.search(r"V.*I.*C.*I.*N.*I.*T.*Y", str(subsequent_row.iloc[1]), re.IGNORECASE)
                third_column_match = re.search(r"C.*L.*A.*S.*S.*I.*F.*I.*C.*A.*T.*I.*O.*N", str(subsequent_row.iloc[2]), re.IGNORECASE | re.DOTALL)
                fourth_column_match = re.search(r"ZV.*SQ.*M", str(subsequent_row.iloc[3]), re.IGNORECASE)
    
                # Set the flag if all columns match the criteria
                if first_column_match and second_column_match and third_column_match and fourth_column_match:
                    additional_info_flag = True
                    # Skip to the row after the current match
                    index += offset
                    break  # No need to check further if a match is found

    # Call the function only if all conditions are true
    if province_match_flag and city_match_flag and barangay_match_flag and additional_info_flag:
        print(f"Province match found in row {index}: {province_value}")
        print(f"City / Municipality match found in row {index + 1}: {city_value}")
        print(f"Barangay / Zone match found in row {index + 2}: {barangay_value}")
        print(f"Additional information match found in row {index + offset}")
        # test_function()
        count += 1
        print(f'{count}\n\n')
        
    # if province_match_flag and (province_match_flag or city_match_flag or barangay_match_flag or additional_info_flag) and not (province_match_flag and city_match_flag and barangay_match_flag and additional_info_flag):
    #     print(f"Condition check in row {index}:")
    #     print(f"  Province match: {'True' if province_match_flag else 'False'}")
    #     print(f"  City/Municipality match: {'True' if city_match_flag else 'False'}")
    #     print(f"  Barangay/Zone match: {'True' if barangay_match_flag else 'False'}")
    #     print(f"  Additional information match: {'True' if additional_info_flag else 'False'}")
    #     count += 1
    #     print(count)

    index += 1
print(count)

Province match found in row 124: :ILOCOS NORTE
City / Municipality match found in row 125: :LAOAG CITY
Barangay / Zone match found in row 126: / Zone :NO. 1 - SAN LORENZO (POBLACION)DO No. 047 - 2023
Additional information match found in row 128
1


Province match found in row 166: :ILOCOS NORTE
City / Municipality match found in row 167: / Municipality :LAOAG CITY
Barangay / Zone match found in row 168: / Zone :NO. 1 - SAN LORENZO (POBLACION) -- continuedDO No. 047 - 2023
Additional information match found in row 170
2


Province match found in row 191: :ILOCOS NORTE
City / Municipality match found in row 192: / Municipality :LAOAG CITY
Barangay / Zone match found in row 193: / Zone :NO. 1 - SAN LORENZO (POBLACION) -- continued
Additional information match found in row 195
3


Province match found in row 222: :ILOCOS NORTE
City / Municipality match found in row 223: / Municipality :LAOAG CITY
Barangay / Zone match found in row 224: / Zone :NO. 2 - SANTA JOAQUINA (POBLACION)DO No. 047 

In [21]:
def test_function():
    print("All conditions met! Function called.\n")

def clean_value(value):
    value = re.sub(r"^\s*:\s*", "", value.strip())
    value = re.sub(r"(DO No|Effectivity Date)\s*.*", "", value, flags=re.IGNORECASE).strip() # do i add --continued here?
    return value

In [22]:
index = 0
count = 0
while index < len(df):
    row = df.iloc[index]
    combined_row = ''.join(map(str, row.dropna())).strip()
    
    # Initialize flags
    province_match_flag = False
    city_match_flag = False
    barangay_match_flag = False
    additional_info_flag = False
    
    # Search for "Province :" pattern
    province_pattern = re.search(r"province\s*(.*)", combined_row, re.IGNORECASE)
    if province_pattern:
        province_match_flag = True
        province_value = province_pattern.group(1)  # Capture the text after "Province :"
        province_value = clean_value(province_value)
        
        print(f"Province match found in row {index}: {province_value}")
    
    city_value = None    
    # Check if the next row has "City" or "Municipality"
    if index + 1 < len(df):
        next_row = ''.join(map(str, df.iloc[index + 1].dropna())).strip()
        cleaned_row = clean_value(next_row)
        city_pattern = re.search(r"(?:City|Municipality)(?:\s*/\s*(?:City|Municipality))?\s*[:\s]?\s*(.+)", cleaned_row, re.IGNORECASE)
        # city_pattern = re.search(r"(?:City|Municipality|Municipality\s*/\s*City|City\s*/\s*Municipality)\s*[:\s]?\s*(\w+)", next_row, re.IGNORECASE)
        if city_pattern:
            city_match_flag = True
            city_value = city_pattern.group(1) # Capture the text after "City :" or "Municipality :"
            # city_value = clean_value(city_value)
            
            print(f"City / Municipality match found in row {index + 1}: {city_value}")
            
    barangay_value = None
    # Check if the row after that has "Barangay" or "Zone"
    if index + 2 < len(df):
        following_row = ''.join(map(str, df.iloc[index + 2].dropna())).strip()
        cleaned_row = clean_value(following_row)
        barangay_pattern = re.search(r"(?:Barangay|Zone)(?:\s*/\s*(?:Barangay|Zone))?\s*[:\s]?\s*(.+)", cleaned_row, re.IGNORECASE)
        if barangay_pattern:
            barangay_match_flag = True
            barangay_value = barangay_pattern.group(1).strip()  # Capture the text after "Barangay :" or "Zone :"
            # barangay_value = clean_value(barangay_value)
            
            print(f"Barangay / Zone match found in row {index + 2}: {barangay_value}")
    
    # Only check for additional information if the primary geographic data is found
    if province_match_flag and city_match_flag and barangay_match_flag:
        # Check specific columns for additional information in the next 3 rows
        for offset in range(3, 6):
            if index + offset < len(df):
                subsequent_row = df.iloc[index + offset]
                
                # Check each column independently
                first_column_match = re.search(r"(STREET NAME|SUBDIVISION|CONDOMINIUM)", str(subsequent_row.iloc[0]), re.IGNORECASE)
                second_column_match = re.search(r"V.*I.*C.*I.*N.*I.*T.*Y", str(subsequent_row.iloc[1]), re.IGNORECASE)
                third_column_match = re.search(r"C.*L.*A.*S.*S.*I.*F.*I.*C.*A.*T.*I.*O.*N", str(subsequent_row.iloc[2]), re.IGNORECASE | re.DOTALL)
                fourth_column_match = re.search(r"ZV.*SQ.*M", str(subsequent_row.iloc[3]), re.IGNORECASE)
    
                # Set the flag if all columns match the criteria
                if first_column_match and second_column_match and third_column_match and fourth_column_match:
                    additional_info_flag = True
                    # Skip to the row after the current match
                    index += offset
                    
                    print(f"Additional information match found in row {index + offset}")
                    
                    break  # No need to check further if a match is found
                    

    # Call the function only if all conditions are true
    if province_match_flag and city_match_flag and barangay_match_flag and additional_info_flag:
        test_function()
        count += 1
        print(f'{count}\n\n')
        
    index += 1
print(count)


City / Municipality match found in row 15: ,  ILOCOS NORTE,  REVENUE  REGION  NO.  1 - CALASIAO,
Barangay / Zone match found in row 22: s  or  areas  and   shall
Barangay / Zone match found in row 23: or   areas,    subject  to   automatic
City / Municipality match found in row 31: , Ilocos Norte, Revenue Region No. 1 - Calasiao, Pangasinan.
City / Municipality match found in row 36: assessor and (2) the gross selling
Barangay / Zone match found in row 108: sA45  Kangkong
City / Municipality match found in row 118: , ILOCOS NORTE
Province match found in row 120: ILOCOS NORTE
City / Municipality match found in row 121: LAOAG CITY
Barangay / Zone match found in row 122: NO. 1 - SAN LORENZO (POBLACION)
Additional information match found in row 128
All conditions met! Function called.

1


City / Municipality match found in row 160: , ILOCOS NORTE
Province match found in row 162: ILOCOS NORTE
City / Municipality match found in row 163: LAOAG CITY
Barangay / Zone match found in row 164: NO.

In [23]:
#TODO: Ask if remove * in classifcaiton and -- contineu in street or smth
index = 0
count = 0
new_df = pd.DataFrame(columns=['Province', 'City/Municipality', 'Barangay', 'Street/Subdivision', 'Vicinity', 'Classification', 'ZV/SQM'])
while index < len(df):
# while count < 3:
    row = df.iloc[index]
    combined_row = ''.join(map(str, row.dropna())).strip()
    
    # Initialize flags
    province_match_flag = False
    city_match_flag = False
    barangay_match_flag = False
    additional_info_flag = False
    
    # Search for "Province :" pattern
    province_pattern = re.search(r"province\s*(.*)", combined_row, re.IGNORECASE)
    if province_pattern:
        province_match_flag = True
        province_value = province_pattern.group(1)  # Capture the text after "Province :"
        province_value = clean_value(province_value)
        
        print(f"Province match found in row {index}: {province_value}")
    
    city_value = None    
    # Check if the next row has "City" or "Municipality"
    if index + 1 < len(df):
        next_row = ''.join(map(str, df.iloc[index + 1].dropna())).strip()
        cleaned_row = clean_value(next_row)
        city_pattern = re.search(r"(?:City|Municipality)(?:\s*/\s*(?:City|Municipality))?\s*[:\s]?\s*(.+)", cleaned_row, re.IGNORECASE)
        # city_pattern = re.search(r"(?:City|Municipality|Municipality\s*/\s*City|City\s*/\s*Municipality)\s*[:\s]?\s*(\w+)", next_row, re.IGNORECASE)
        if city_pattern:
            city_match_flag = True
            city_value = city_pattern.group(1) # Capture the text after "City :" or "Municipality :"
            # city_value = clean_value(city_value)
            
            print(f"City / Municipality match found in row {index + 1}: {city_value}")
            
    barangay_value = None
    # Check if the row after that has "Barangay" or "Zone"
    if index + 2 < len(df):
        following_row = ''.join(map(str, df.iloc[index + 2].dropna())).strip()
        cleaned_row = clean_value(following_row)
        barangay_pattern = re.search(r"(?:Barangay|Zone)(?:\s*/\s*(?:Barangay|Zone))?\s*[:\s]?\s*(.+)", cleaned_row, re.IGNORECASE)
        if barangay_pattern:
            barangay_match_flag = True
            barangay_value = barangay_pattern.group(1).strip()  # Capture the text after "Barangay :" or "Zone :"
            # barangay_value = clean_value(barangay_value)
            
            print(f"Barangay / Zone match found in row {index + 2}: {barangay_value}")
    
    # Only check for additional information if the primary geographic data is found
    if province_match_flag and city_match_flag and barangay_match_flag:
        # Check specific columns for additional information in the next 3 rows
        for offset in range(3, 6):
            if index + offset < len(df):
                subsequent_row = df.iloc[index + offset]
                
                # Initialize variables to store the column indices of matches
                street_name_index = None
                vicinity_index = None
                classification_index = None
                zv_sq_m_index = None
    
                # Iterate over each column in the row
                for col_index in range(len(subsequent_row)):
                    cell_value = str(subsequent_row.iloc[col_index])
                    
                    # Check each column independently
                    if re.search(r"(STREET NAME|SUBDIVISION|CONDOMINIUM)", cell_value, re.IGNORECASE):
                        street_name_index = col_index  # Store the index of the match for street name/subdivision/condominium
                    
                    if re.search(r"V.*I.*C.*I.*N.*I.*T.*Y", cell_value, re.IGNORECASE):
                        vicinity_index = col_index  # Store the index of the match for vicinity
                    
                    if re.search(r"C.*L.*A.*S.*S.*I.*F.*I.*C.*A.*T.*I.*O.*N", cell_value, re.IGNORECASE | re.DOTALL):
                        classification_index = col_index  # Store the index of the match for classification
                    
                    if re.search(r"ZV.*SQ.*M", cell_value, re.IGNORECASE):
                        zv_sq_m_index = col_index  # Store the index of the match for ZV SQ M
    
                # If all matches are found, set the additional_info_flag to True
                if street_name_index is not None and vicinity_index is not None and classification_index is not None and zv_sq_m_index is not None:
                    additional_info_flag = True
                    # Print the matched indices for debugging purposes
                    print(f"Additional information match found in row {index + offset}")
                    print(f"STREET NAME found in column {street_name_index}")
                    print(f"VICINITY found in column {vicinity_index}")
                    print(f"CLASSIFICATION found in column {classification_index}")
                    print(f"ZV SQ M found in column {zv_sq_m_index}")
                    
                    # Skip to the row after the current match
                    index += offset + 1
                    
                    break  # No need to check further if a match is found

    # Call the function only if all conditions are true
    if province_match_flag and city_match_flag and barangay_match_flag and additional_info_flag:
        test_function()
        count += 1
        print(f'{count}\n\n')
        
        # start reading here: age_counter
        age = 0
        col1_holder = None
        vicinity_holder = None 
        while age < 2:
            row = df.iloc[index]
            
            col1 = row.iloc[street_name_index]
            vicinity = row.iloc[vicinity_index]
            classification = row.iloc[classification_index]
            zv = row.iloc[zv_sq_m_index]
            test = [col1, vicinity, classification, zv]
            # print(f"base row: {test}")
            
            if (pd.isnull(classification) or classification.strip() == '') and pd.isnull(zv):
                age += 1
                index += 1
                # print("aging")
                continue
            
            age = 0
            
            if pd.isnull(col1) or col1.strip() == '':
                col1 = col1_holder
            if pd.isnull(vicinity) or vicinity.strip() == '':
                vicinity = vicinity_holder
            
            col1_holder = col1
            vicinity_holder = vicinity
            
            new_df.loc[len(new_df)] = [province_value, city_value, barangay_value, col1, vicinity, classification, zv]
            index += 1
        
        index += 1 
        continue
        
    index += 1
print(count)


City / Municipality match found in row 15: ,  ILOCOS NORTE,  REVENUE  REGION  NO.  1 - CALASIAO,
Barangay / Zone match found in row 22: s  or  areas  and   shall
Barangay / Zone match found in row 23: or   areas,    subject  to   automatic
City / Municipality match found in row 31: , Ilocos Norte, Revenue Region No. 1 - Calasiao, Pangasinan.
City / Municipality match found in row 36: assessor and (2) the gross selling
Barangay / Zone match found in row 108: sA45  Kangkong
City / Municipality match found in row 118: , ILOCOS NORTE
Province match found in row 120: ILOCOS NORTE
City / Municipality match found in row 121: LAOAG CITY
Barangay / Zone match found in row 122: NO. 1 - SAN LORENZO (POBLACION)
Additional information match found in row 124
STREET NAME found in column 0
VICINITY found in column 1
CLASSIFICATION found in column 2
ZV SQ M found in column 3
All conditions met! Function called.

1


City / Municipality match found in row 160: , ILOCOS NORTE
Province match found in row 

In [24]:
new_df

,Province,City/Municipality,Barangay,Street/Subdivision,Vicinity,Classification,ZV/SQM
0,ILOCOS NORTE,LAOAG CITY,NO. 1 - SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),D SAMONTE ST. - PRIMO LAZARO ST(P. BURGOS ST),RR,7300.0
1,ILOCOS NORTE,LAOAG CITY,NO. 1 - SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),PRIMO LAZARO ST(P BURGOS ST)- RIZAL ST,CR*,8700.0
2,ILOCOS NORTE,LAOAG CITY,NO. 1 - SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),PRIMO LAZARO ST(P BURGOS ST)- RIZAL ST,RR,7300.0
3,ILOCOS NORTE,LAOAG CITY,NO. 1 - SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),D SAMONTE ST. - RIZAL ST,RR,7300.0
4,ILOCOS NORTE,LAOAG CITY,NO. 1 - SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),INTERIOR,RR,5500.0
...,...,...,...,...,...,...,...
14594,ILOCOS NORTE,VINTAR,ISIC-ISIC,ALL LOTS,ALONG THE BORDER OF THE,RR,190
14595,ILOCOS NORTE,VINTAR,ISIC-ISIC,ALL LOTS,MOUNTAIN,A3,48
14596,ILOCOS NORTE,VINTAR,ISIC-ISIC,ALL LOTS,MOUNTAIN,A15,53
14597,ILOCOS NORTE,VINTAR,ISIC-ISIC,ALL LOTS,MOUNTAIN,A16*,63


In [25]:
row = df.iloc[135]
# if pd.isnull(classification) and (pd.isnull(zv) or zv.strip() == ''):
#     print("aging")

col1 = row.iloc[street_name_index]
vicinity = row.iloc[vicinity_index]
classification = row.iloc[classification_index]
zv = row.iloc[zv_sq_m_index]
test = [col1, vicinity, classification, zv]

pd.isnull(zv)

True

In [26]:
df.loc[125]

0    NaN
1    NaN
2    NaN
3    NaN
Name: 125, dtype: object

In [27]:
following_row = ''.join(map(str, df.iloc[25184].dropna())).strip()
barangay_pattern = re.search(r"(?:Barangay|Zone)(?:\s*/\s*(?:Barangay|Zone))?\s*[:\s]?\s*(\w+)", following_row, re.IGNORECASE)
if barangay_pattern:
    barangay_match_flag = True
    barangay_value = barangay_pattern.group(1).strip()  # Capture the text after "Barangay :" or "Zone :"
    print(f"Barangay / Zone match found in row {index + 2}: {barangay_value}")
    barangay_value = clean_value(barangay_value)
    print(f"Barangay / Zone match found in row {index + 2}: {barangay_value}")

Barangay / Zone match found in row 25348: CABAYODO
Barangay / Zone match found in row 25348: CABAYODO


In [28]:
following_row

'Barangay / Zone :CABAYODO No. 047 - 2023'

In [29]:
clean_value(following_row)

'Barangay / Zone :CABAYO'

In [30]:
testtest.iloc[0]

NameError: name 'testtest' is not defined

In [ ]:
re.search(r"C\s*L\s*A\s*S\s*S\s*I\s*F\s*I\s*C\s*A\s*T\s*I\s*O\s*N", str(testtest.iloc[2]), re.IGNORECASE)

In [ ]:
import re

joined_row = "Country: Philippines, Province : Quezon, City: Quezon City"

pattern = r"province\s*:"

match = re.search(pattern, joined_row, re.IGNORECASE)

if match:
    print("Match found:", match.group())
else:
    print("No match found.")


In [ ]:
def main(df):
    #TODO: Ask if remove * in classifcaiton and -- contineu in street or smth
    index = 0
    count = 0
    new_df = pd.DataFrame(columns=['Province', 'City/Municipality', 'Barangay', 'Street/Subdivision', 'Vicinity', 'Classification', 'ZV/SQM'])
    while index < len(df):
    # while count < 3:
        row = df.iloc[index]
        combined_row = ''.join(map(str, row.dropna())).strip()
        
        # Initialize flags
        province_match_flag = False
        city_match_flag = False
        barangay_match_flag = False
        additional_info_flag = False
        
        # Search for "Province :" pattern
        province_pattern = re.search(r"province\s*(.*)", combined_row, re.IGNORECASE)
        if province_pattern:
            province_match_flag = True
            province_value = province_pattern.group(1)  # Capture the text after "Province :"
            province_value = clean_value(province_value)
            
            print(f"Province match found in row {index}: {province_value}")
        
        city_value = None    
        # Check if the next row has "City" or "Municipality"
        if index + 1 < len(df):
            next_row = ''.join(map(str, df.iloc[index + 1].dropna())).strip()
            cleaned_row = clean_value(next_row)
            city_pattern = re.search(r"(?:City|Municipality)(?:\s*/\s*(?:City|Municipality))?\s*[:\s]?\s*(.+)", cleaned_row, re.IGNORECASE)
            # city_pattern = re.search(r"(?:City|Municipality|Municipality\s*/\s*City|City\s*/\s*Municipality)\s*[:\s]?\s*(\w+)", next_row, re.IGNORECASE)
            if city_pattern:
                city_match_flag = True
                city_value = city_pattern.group(1) # Capture the text after "City :" or "Municipality :"
                # city_value = clean_value(city_value)
                
                print(f"City / Municipality match found in row {index + 1}: {city_value}")
                
        barangay_value = None
        # Check if the row after that has "Barangay" or "Zone"
        if index + 2 < len(df):
            following_row = ''.join(map(str, df.iloc[index + 2].dropna())).strip()
            cleaned_row = clean_value(following_row)
            barangay_pattern = re.search(r"(?:Barangay|Zone)(?:\s*/\s*(?:Barangay|Zone))?\s*[:\s]?\s*(.+)", cleaned_row, re.IGNORECASE)
            if barangay_pattern:
                barangay_match_flag = True
                barangay_value = barangay_pattern.group(1).strip()  # Capture the text after "Barangay :" or "Zone :"
                # barangay_value = clean_value(barangay_value)
                
                print(f"Barangay / Zone match found in row {index + 2}: {barangay_value}")
        
        # Only check for additional information if the primary geographic data is found
        if province_match_flag and city_match_flag and barangay_match_flag:
            # Check specific columns for additional information in the next 3 rows
            for offset in range(3, 6):
                if index + offset < len(df):
                    subsequent_row = df.iloc[index + offset]
                    
                    # Initialize variables to store the column indices of matches
                    street_name_index = None
                    vicinity_index = None
                    classification_index = None
                    zv_sq_m_index = None
        
                    # Iterate over each column in the row
                    for col_index in range(len(subsequent_row)):
                        cell_value = str(subsequent_row.iloc[col_index])
                        
                        # Check each column independently
                        if re.search(r"(STREET NAME|SUBDIVISION|CONDOMINIUM)", cell_value, re.IGNORECASE):
                            street_name_index = col_index  # Store the index of the match for street name/subdivision/condominium
                        
                        if re.search(r"V.*I.*C.*I.*N.*I.*T.*Y", cell_value, re.IGNORECASE):
                            vicinity_index = col_index  # Store the index of the match for vicinity
                        
                        if re.search(r"C.*L.*A.*S.*S.*I.*F.*I.*C.*A.*T.*I.*O.*N", cell_value, re.IGNORECASE | re.DOTALL):
                            classification_index = col_index  # Store the index of the match for classification
                        
                        if re.search(r"ZV.*SQ.*M", cell_value, re.IGNORECASE):
                            zv_sq_m_index = col_index  # Store the index of the match for ZV SQ M
        
                    # If all matches are found, set the additional_info_flag to True
                    if street_name_index is not None and vicinity_index is not None and classification_index is not None and zv_sq_m_index is not None:
                        additional_info_flag = True
                        # Print the matched indices for debugging purposes
                        print(f"Additional information match found in row {index + offset}")
                        print(f"STREET NAME found in column {street_name_index}")
                        print(f"VICINITY found in column {vicinity_index}")
                        print(f"CLASSIFICATION found in column {classification_index}")
                        print(f"ZV SQ M found in column {zv_sq_m_index}")
                        
                        # Skip to the row after the current match
                        index += offset + 1
                        
                        break  # No need to check further if a match is found
    
        # Call the function only if all conditions are true
        if province_match_flag and city_match_flag and barangay_match_flag and additional_info_flag:
            test_function()
            count += 1
            print(f'{count}\n\n')
            
            # start reading here: age_counter
            age = 0
            col1_holder = None
            vicinity_holder = None 
            while age < 2:
                row = df.iloc[index]
                
                col1 = row.iloc[street_name_index]
                vicinity = row.iloc[vicinity_index]
                classification = row.iloc[classification_index]
                zv = row.iloc[zv_sq_m_index]
                test = [col1, vicinity, classification, zv]
                # print(f"base row: {test}")
                
                if (pd.isnull(classification) or classification.strip() == '') and pd.isnull(zv):
                    age += 1
                    index += 1
                    # print("aging")
                    continue
                
                age = 0
                
                if pd.isnull(col1) or col1.strip() == '':
                    col1 = col1_holder
                if pd.isnull(vicinity) or vicinity.strip() == '':
                    vicinity = vicinity_holder
                
                col1_holder = col1
                vicinity_holder = vicinity
                
                new_df.loc[len(new_df)] = [province_value, city_value, barangay_value, col1, vicinity, classification, zv]
                index += 1
            
            index += 1 
            continue
            
        index += 1
    print(count)
    return new_df

In [ ]:
def xls_to_df(filename):
    test = pd.ExcelFile("data/" + filename)
    sheet_names = test.sheet_names
    last_sheet_name = None
    
    # Extract sheet names that start with 'sheet'
    sheet_names = sorted([name for name in sheet_names if name.strip().lower().startswith('sheet')],
                         key=lambda name: int(re.search(r'\d+', name).group()))
    
    if sheet_names:
        last_sheet_name = sheet_names[-1]
    
    if last_sheet_name:
        df = pd.read_excel("data/" + filename, sheet_name=last_sheet_name, header=None)
        return df
    else:
        print(f"No matching sheets found in {filename}")


In [ ]:
excel_files[0]

In [ ]:
xls_to_df(excel_files[1])

In [ ]:
for excel in excel_files[1:]:
    df = xls_to_df(excel)
    processed = main(df)
    processed.to_excel(f"Output/{excel}.xlsx")

In [ ]:
excel_file = pd.ExcelFile("data/" + excel_files[1])
print(excel_file.sheet_names)

In [ ]:
df = pd.read_excel("data/" + excel_files[1], sheet_name='Sheet 3 (DO 044-2022)', header=None)

In [ ]:
df

In [ ]:
test = pd.ExcelFile("data/" + excel_files[1])
sheet_names = test.sheet_names
last_sheet_name = None

In [ ]:
# Extract sheet names that start with 'sheet'
sheet_names = sorted([name for name in sheet_names if name.strip().lower().startswith('sheet')],
                     key=lambda name: int(re.search(r'\d+', name).group()))

In [ ]:
if sheet_names:
    last_sheet_name = sheet_names[-1]

In [ ]:
last_sheet_name

In [ ]:
if last_sheet_name:
    df = pd.read_excel("data/" + file, sheet_name=last_sheet_name, header=None)
else:
    print(f"No matching sheets found in {file}")

In [ ]:
df = pd.read_excel("data/" + excel_files[1], sheet_name='Sheet 3 (DO 044-2022)', header=None)

In [ ]:
x = pd.read_excel("Output/RDO No. 8 - Baguio City.xls.xlsx")

In [ ]:
len(x)

In [31]:
output_files = [f for f in os.listdir("Output")]
for output_file in output_files:
    file = pd.read_excel(f'Output/{output_file}')
    if len(file) < 2:
        print(output_file[:-5])

RDO No. 105 - Surigao City, Surigao del Norte.xls
RDO No. 12 - Lagawe, Ifugao.xlsx
RDO No. 18 - Olongapo City, Zambales.xls
RDO No. 32 - Quiapo-Sampaloc-San Miguel-Sta Mesa.xls
RDO No. 46 - Cainta-Taytay.xls
RDO No. 60 - Lucena City, North Quezon.xls
RDO No. 61 - Gumaca, South Quezon.xls
RDO No. 8 - Baguio City.xls
RDO No. 80 - Mandaue City, Cebu.xls
RDO No. 84 - Tagbilaran City, Bohol.XLS
RDO No. 9 - La Trinidad, Benguet.xlsx


PermissionError: [Errno 13] Permission denied: 'Output/~$RDO No. 2 - Vigan City, Ilocos Sur.xls.xlsx'

In [ ]:
import os

In [ ]:
len(output_files)